<a href="https://colab.research.google.com/github/Chiranjith18/Lonewolf/blob/master/safebite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# Load all 6 CSV files safely
df1 = pd.read_csv("coimbatore.csv", encoding="latin1")
df2 = pd.read_csv("chennai.csv", encoding="latin1")
df3 = pd.read_csv("bangalore.csv", encoding="latin1")
df4 = pd.read_csv("hyderabad.csv", encoding="latin1")
df5 = pd.read_csv("madurai.csv", encoding="latin1")
df6 = pd.read_csv("trichy.csv", encoding="latin1")

# Combine all vertically (not merge)
dataset = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

# Ensure column consistency
dataset = dataset[["No", "City", "Restaurant", "Review"]]

# Renumber the 'No' column sequentially
dataset["No"] = range(1, len(dataset) + 1)

# Save final combined dataset
dataset.to_csv("dataset.csv", index=False, encoding="utf-8-sig")

print("✅ Combined dataset created successfully as dataset.csv")
print(f"Total rows: {len(dataset)}")


✅ Combined dataset created successfully as dataset.csv
Total rows: 500


In [9]:
import pandas as pd

# Replace 'your_dataset.csv' with your actual CSV filename including path if needed
df = pd.read_csv('dataset.csv')

# Verify loading by checking the first few rows
print(df.head())

# Check column names to ensure they loaded correctly
print(df.columns)


   No        City       Restaurant  \
0   1  Coimbatore  Barbeque Nation   
1   2  Coimbatore  Barbeque Nation   
2   3  Coimbatore  Barbeque Nation   
3   4  Coimbatore  Barbeque Nation   
4   5  Coimbatore  Barbeque Nation   

                                              Review  
0  Niyamat Ali his service to us is very good and...  
1                          The service was excellent  
2                                   Vikram Senthil S  
3  They cancelled our billing on easy-diner just ...  
4  Customer service was excellent. Delicious food...  
Index(['No', 'City', 'Restaurant', 'Review'], dtype='object')


In [10]:
import pandas as pd

dataset = pd.read_csv("dataset.csv", encoding="utf-8")

# Keep reviews with at least 4 words
dataset = dataset[dataset["Review"].str.split().str.len() >= 4]

# Remove reviews containing too many question marks
dataset = dataset[~dataset["Review"].str.contains(r"\?{3,}", na=False)]

# Reset numbering
dataset = dataset.reset_index(drop=True)
dataset["No"] = range(1, len(dataset) + 1)

# Save the cleaned dataset
dataset.to_csv("cleaned_dataset.csv", index=False, encoding="utf-8-sig")

print("✅ Cleaned dataset saved as cleaned_dataset.csv")
print(f"Remaining rows: {len(dataset)}")


✅ Cleaned dataset saved as cleaned_dataset.csv
Remaining rows: 450


In [12]:
!pip install transformers


In [14]:
import pandas as pd
from transformers import pipeline

# Load your dataset
df = pd.read_csv('cleaned_dataset.csv')  # replace with your CSV filename

# Initialize sentiment-analysis pipeline (default is distilbert-base-uncased-finetuned-sst-2-english)
sentiment_pipeline = pipeline("sentiment-analysis")

# Function to get sentiment label from HuggingFace model
def get_hf_sentiment(text):
    result = sentiment_pipeline(str(text))[0]
    label = result['label'].lower()  # usually 'positive' or 'negative'
    return label

# Apply the function to your Review column
df['Sentiment'] = df['Review'].apply(get_hf_sentiment)

# Save results to a new CSV
df.to_csv('dataset_with_hf_sentiment.csv', index=False)

print("HuggingFace sentiment analysis complete and saved as dataset_with_hf_sentiment.csv")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


HuggingFace sentiment analysis complete and saved as dataset_with_hf_sentiment.csv


In [18]:
import pandas as pd
from transformers import pipeline

# Load your dataset with reviews
df = pd.read_csv('dataset_with_hf_sentiment.csv')  # replace with your actual file

# Initialize zero-shot classification pipeline with the correct model (CPU mode)
classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli",
    device=-1  # -1 for CPU
)

candidate_labels = [
    "high risk", "medium risk", "low risk",
    "hygiene complaint", "food quality complaint", "service complaint",
    "ambience complaint", "price complaint"
]

batch_size = 16
risk_labels = []

# Batched zero-shot classification
for start in range(0, len(df), batch_size):
    batch_texts = df['Review'].iloc[start:start+batch_size].tolist()
    results = classifier(batch_texts, candidate_labels)
    # Handle single item vs batch output
    if isinstance(results, dict):
        risk_labels.append(results['labels'][0])
    else:
        risk_labels.extend([res['labels'][0] for res in results])

# Add labels to dataframe
df['Risk_Label'] = risk_labels

# Save to CSV
output_file = 'dataset_with_zero_shot_labels.csv'
df.to_csv(output_file, index=False)

print(f"Zero-shot classification complete and saved to {output_file}")


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Zero-shot classification complete and saved to dataset_with_zero_shot_labels.csv


In [26]:
# Assuming 'df' has all columns including zero-shot 'Risk_Label'

# Select only important columns
important_df = df[['City', 'Restaurant', 'Risk_Label']].copy()

# Optionally rename the classification column for clarity
important_df.rename(columns={'Risk_Label': 'Classification'}, inplace=True)

# Save to CSV for reuse
important_df.to_csv('restaurant_classification_summary.csv', index=False)

print("Saved important columns to restaurant_classification_summary.csv")


Saved important columns to restaurant_classification_summary.csv


In [27]:
# Load summary df
import pandas as pd
df = pd.read_csv('restaurant_classification_summary.csv')

# Define ranking order
risk_order = {
    "High Risk": 1,
    "Medium Risk": 2,
    "Low Risk": 3,
    "high risk": 1,
    "medium risk": 2,
    "low risk": 3,
    "food quality complaint": 1,
    "hygiene complaint": 1,
    "service complaint": 2,
    "ambience complaint": 3,
    "price complaint": 3,
    "Unknown": 4,
    # Add other labels if needed
}

# Add rank column based on Classification
df['Risk_Rank'] = df['Classification'].map(risk_order).fillna(99)

# Sort df by City and ascending rank (so High Risk are on top)
df_sorted = df.sort_values(['City', 'Risk_Rank'])


In [34]:
print(df.columns)
print(df.head())


Index(['No', 'City', 'Restaurant', 'Review', 'Sentiment', 'Risk_Label'], dtype='object')
   No        City            Restaurant  \
0   1  Coimbatore       Barbeque Nation   
1   2  Coimbatore       Barbeque Nation   
2   3  Coimbatore       Barbeque Nation   
3   4  Coimbatore       Barbeque Nation   
4   5  Coimbatore  Bulls & Bears Bistro   

                                              Review Sentiment  \
0  Niyamat Ali his service to us is very good and...  positive   
1                          The service was excellent  positive   
2  They cancelled our billing on easy-diner just ...  negative   
3  Customer service was excellent. Delicious food...  positive   
4  I don't know what Harshita saw in this place, ...  positive   

               Risk_Label  
0      ambience complaint  
1      ambience complaint  
2  food quality complaint  
3  food quality complaint  
4  food quality complaint  


In [35]:
import pandas as pd

# Load your zero-shot labelled dataset
df = pd.read_csv('dataset_with_zero_shot_labels.csv')

# Define which labels belong to which risk category
high_risk_labels = ["high risk", "hygiene complaint", "food quality complaint", "health violation", "inspection required"]
medium_risk_labels = ["medium risk", "service complaint", "ambience complaint"]
low_risk_labels = ["low risk", "price complaint", "other"]

# Count the Risk_Label per Restaurant
risk_counts = df.groupby(['Restaurant', 'Risk_Label']).size().unstack(fill_value=0)

# Ensure all referenced columns exist
for label in high_risk_labels + medium_risk_labels + low_risk_labels:
    if label not in risk_counts.columns:
        risk_counts[label] = 0

# Sum counts per risk category
risk_counts['High_Risk_Total'] = risk_counts[high_risk_labels].sum(axis=1)
risk_counts['Medium_Risk_Total'] = risk_counts[medium_risk_labels].sum(axis=1)
risk_counts['Low_Risk_Total'] = risk_counts[low_risk_labels].sum(axis=1)

# Calculate total reviews per restaurant
risk_counts['Total'] = risk_counts['High_Risk_Total'] + risk_counts['Medium_Risk_Total'] + risk_counts['Low_Risk_Total']

# Calculate proportions
risk_counts['High_Risk_Proportion'] = risk_counts['High_Risk_Total'] / risk_counts['Total']
risk_counts['Medium_Risk_Proportion'] = risk_counts['Medium_Risk_Total'] / risk_counts['Total']

# Define function to classify final risk
def final_risk(row):
    if row['High_Risk_Proportion'] > 0.3:
        return 'High Risk'
    elif row['Medium_Risk_Proportion'] > 0.15:
        return 'Medium Risk'
    else:
        return 'Low Risk'

risk_counts['Final_Risk_Category'] = risk_counts.apply(final_risk, axis=1)

# Keep only required columns and reset index to get restaurant names
final_risk_df = risk_counts[['Final_Risk_Category']].reset_index()

print(final_risk_df.head())

# Save final classification per restaurant
final_risk_df.to_csv('restaurant_final_risk_classification.csv', index=False)

print("Final restaurant risk classification saved.")


Risk_Label                 Restaurant Final_Risk_Category
0                          404 by TOF           High Risk
1           A2B - Adyar Ananda Bhavan         Medium Risk
2                  Ammikal Restaurant         Medium Risk
3                    Amsavalli Bhavan           High Risk
4                   Annapoorna Mithai         Medium Risk
Final restaurant risk classification saved.


In [36]:
final_risk_df.to_csv('restaurant_final_risk_classification.csv', index=False)

print("Final restaurant risk classification saved.")


Final restaurant risk classification saved.


In [37]:
import pandas as pd

# Load your zero-shot labelled dataset
df = pd.read_csv('dataset_with_zero_shot_labels.csv')

# Define which labels belong to which risk category
high_risk_labels = ["high risk", "hygiene complaint", "food quality complaint", "health violation", "inspection required"]
medium_risk_labels = ["medium risk", "service complaint", "ambience complaint"]
low_risk_labels = ["low risk", "price complaint", "other"]

# Count the Risk_Label per City+Restaurant
risk_counts = df.groupby(['City', 'Restaurant', 'Risk_Label']).size().unstack(fill_value=0)

# Ensure all referenced columns exist
for label in high_risk_labels + medium_risk_labels + low_risk_labels:
    if label not in risk_counts.columns:
        risk_counts[label] = 0

# Sum counts per risk category
risk_counts['High_Risk_Total'] = risk_counts[high_risk_labels].sum(axis=1)
risk_counts['Medium_Risk_Total'] = risk_counts[medium_risk_labels].sum(axis=1)
risk_counts['Low_Risk_Total'] = risk_counts[low_risk_labels].sum(axis=1)

risk_counts['Total'] = risk_counts['High_Risk_Total'] + risk_counts['Medium_Risk_Total'] + risk_counts['Low_Risk_Total']

# Calculate proportions
risk_counts['High_Risk_Proportion'] = risk_counts['High_Risk_Total'] / risk_counts['Total']
risk_counts['Medium_Risk_Proportion'] = risk_counts['Medium_Risk_Total'] / risk_counts['Total']

def final_risk(row):
    if row['High_Risk_Proportion'] > 0.3:
        return 'High Risk'
    elif row['Medium_Risk_Proportion'] > 0.15:
        return 'Medium Risk'
    else:
        return 'Low Risk'

risk_counts['Final_Risk_Category'] = risk_counts.apply(final_risk, axis=1)

# Reset index to get City and Restaurant as columns
final_df = risk_counts.reset_index()[['City', 'Restaurant', 'Final_Risk_Category']]

print(final_df.head())

final_df.to_csv('restaurant_final_risk_classification.csv', index=False)
print("Final risk classification (city, restaurant, label) saved.")


Risk_Label       City                Restaurant Final_Risk_Category
0           Bangalore                404 by TOF           High Risk
1           Bangalore      Bier Loft Brewing Co         Medium Risk
2           Bangalore                 Go Native         Medium Risk
3           Bangalore                    Homiga         Medium Risk
4           Bangalore  Ironhill Kanakapura road         Medium Risk
Final risk classification (city, restaurant, label) saved.


In [39]:
import pandas as pd

# Load your zero-shot labelled dataset
df = pd.read_csv('dataset_with_zero_shot_labels.csv')

high_risk_labels = ["high risk", "hygiene complaint", "food quality complaint", "health violation", "inspection required"]
medium_risk_labels = ["medium risk", "service complaint", "ambience complaint"]
low_risk_labels = ["low risk", "price complaint", "other"]

risk_counts = df.groupby(['City', 'Restaurant', 'Risk_Label']).size().unstack(fill_value=0)

for label in high_risk_labels + medium_risk_labels + low_risk_labels:
    if label not in risk_counts.columns:
        risk_counts[label] = 0

risk_counts['High_Risk_Total'] = risk_counts[high_risk_labels].sum(axis=1)
risk_counts['Medium_Risk_Total'] = risk_counts[medium_risk_labels].sum(axis=1)
risk_counts['Low_Risk_Total'] = risk_counts[low_risk_labels].sum(axis=1)

risk_counts['Total'] = risk_counts['High_Risk_Total'] + risk_counts['Medium_Risk_Total'] + risk_counts['Low_Risk_Total']

risk_counts['High_Risk_Proportion'] = risk_counts['High_Risk_Total'] / risk_counts['Total']
risk_counts['Medium_Risk_Proportion'] = risk_counts['Medium_Risk_Total'] / risk_counts['Total']

def final_risk(row):
    if row['High_Risk_Proportion'] > 0.3:
        return 'High Risk'
    elif row['Medium_Risk_Proportion'] > 0.15:
        return 'Medium Risk'
    else:
        return 'Low Risk'

risk_counts['Category'] = risk_counts.apply(final_risk, axis=1)

final_df = risk_counts.reset_index()[['City', 'Restaurant', 'Category']]



final_df.to_csv('restaurant_final_risk_classification.csv', index=False)
print("Saved as 'restaurant_final_risk_classification.csv' with columns: City, Restaurant, Category, category")
print(final_df.head())


Saved as 'restaurant_final_risk_classification.csv' with columns: City, Restaurant, Category, category
Risk_Label       City                Restaurant     Category
0           Bangalore                404 by TOF    High Risk
1           Bangalore      Bier Loft Brewing Co  Medium Risk
2           Bangalore                 Go Native  Medium Risk
3           Bangalore                    Homiga  Medium Risk
4           Bangalore  Ironhill Kanakapura road  Medium Risk
